<a href="https://colab.research.google.com/github/yeesem/Advanced-Tensorflow-Specialization/blob/main/Image_Classification_and_Object_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os,re,time,json
import PIL.Image,PIL.ImageFont,PIL.ImageDraw
import numpy as np

import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds

# Visualization Utilities

In [3]:
im_width = 75
im_height = 75
use_normalized_coordinates = True

def draw_bounding_boxes_on_image_array(
    image,
    boxes,
    color = [],
    thickness = 1,
    display_str_list = ()
):
  # The input image, which is a NumPy array, is converted into a PIL
  # (Python Imaging Library) image object using PIL.Image.fromarray.
  image_pil = PIL.Image.fromaarray(image)
  # A new RGBA image is created with the same size as the original image.
  # The "RGBA" mode means that the new image will have Red, Green, Blue, and
  # Alpha (transparency) channels.
  rgbimg = PIL.Image.new("RGBA",image_pil.size)
  # Paste the Original Image onto the RGBA Image:
  rgbimg.paste(image_pil)
  draw_bounding_boxes_on_image(rgbimg,boxes,color,thickness,
                               display_str_list)
  return np.array(rgbimg)

def draw_bounding_boxes_on_image(
    image,
    boxes,
    color = [],
    thickness = 1,
    display_str_list = ()
):
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape)!=2 or boxes_shape[1] != 4:
    raise ValueError("Input must be of size [N,4]")
  for i in range(boxes_shape[0]):
    draw_bounding_box_on_image(image,boxes[i,1],boxes[i,0],boxes[i,3],
                               boxes[i,2],color[i],thickness,display_str_list[i])

def draw_bounding_box_on_image(
    image,
    ymin,
    xmin,
    ymax,
    xmax,
    color = 'red',
    thickness = 1,
    display_str = None,
    use_normalized_coordinates = True
):
  draw = PIL.ImageDraw.Draw(image)
  im_width,im_height = image.size
  if use_normalized_coordinates:
    (left,right,top,bottom) = (xmin * im_width,xmax * im_width,
                               ymin* im_height,ymax * im_height)
  else:
    (left,right,top,bottom) = (xmin,xmax,ymin,ymax)

  draw.line([(left,top),(left,bottom),(right,bottom),
             (right,top),(left,top)],width = thickness,fill = color)


In [ ]:
plt.rc('image',cmap='gray')
plt.rc('grid',linewidth = 0)
plt.rc('xtick',top = False,bottom = False,labelsize = 'large')
plt.rc('ytick',left = False,right = False,labelsize = 'large')
plt.rc('axes',facecolor = 'F8F8F8',titlesize = 'large',edgecolor = 'white')
plt.rc('text',color = 'a8151a')
plt.rc('figure', facecolor='F0F0F0')# Matplotlib fonts
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")

def dataset_to_numpy_util(training_dataset,validation_dataset,N):

  # get one batch from each: 10000 validation digits, N training digits
  batch_train_ds = training_dataset.unbatch().batch(N)

  # Eager execution : Loop through datasets normally
  if tf.executing_eagerly():
    for validation_digits, (validation_labels,validation_bboxes) in validation_dataset:
      validation_digits = validation_digits.numpy()
      validation_labels = validation_labels.numpy()
      validation_bboxes = validation_bboxes.numpy()
      break
    for training_digits, (training_labels,training_bboxes) in batch_train_ds:
      training_digits = training_digits.numpy()
      training_labels = training_labels.numpy()
      training_bboxes = training_bboxes.numpy()
      break

    # these were one-hot encoded in the dataset
    validation_labels = np.argmax(validation_labels,axis = 1)
    training_labels = np.argmax(training_labels,axis = 1)

    return (training_digits, training_labels, training_bboxes,
          validation_digits, validation_labels, validation_bboxes)

